电离层校正的东西不是完全没写，只是不算完善。

In [1]:
import sys
sys.path.append("..")
import os
os.chdir("..")
from netCDF4 import Dataset
import numpy as np
from functions.image import wgs2ecs, ecs2wgs
from functions.trans import cos_vectors


class Ion(object):
    def __init__(self, path) -> None:
        data = Dataset(path)
        self.data = np.array(data['tecmap'])

    def find(self, sod, lon, lat):
        i = np.int32(sod // 900)
        j = np.int32(90 - lat)
        k = np.int32(lon + 180)
        return self.data[i, j, k]

    def cal_delay(self, sod, sp, lon, lat, h, f):
        height = 450e3
        tp = wgs2ecs(lon, lat, h)
        cos_zenith = cos_vectors(tp, sp - tp)
        length = height / cos_zenith
        dp = sp - tp
        pierces = dp * length / np.linalg.norm(dp, axis=0) + tp

        plon, plat, _ = ecs2wgs(pierces[0], pierces[1], pierces[2])
        tec = self.find(sod, plon, plat)
        return 40.3e16 * tec / (f**2*cos_vectors(pierces, sp-wgs2ecs(lon, lat, h)))
